<a href="https://colab.research.google.com/github/andriasmagno/Day-Trading-Algorithms-/blob/main/backtesting_bollinger_band_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ccxt
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# Configurar o cliente Binance
exchange = ccxt.binance()

# Função para obter candles de 5 minutos nos últimos 3 meses
def get_candles_last_3_months(symbol, timeframe='5m'):
    """
    Obtém candles de 5 minutos para os últimos 3 meses de um símbolo específico.
    """
    end_time = datetime.utcnow()
    start_time = end_time - timedelta(days=90)  # Período de 3 meses
    since = int(start_time.timestamp() * 1000)  # Converter para milissegundos
    all_candles = []

    while since < int(end_time.timestamp() * 1000):
        # Buscar candles
        candles = exchange.fetch_ohlcv(symbol, timeframe, since=since, limit=1000)
        if not candles:
            break
        all_candles.extend(candles)
        since = candles[-1][0] + 1  # Atualizar 'since' para evitar duplicatas

    # Converter para DataFrame
    df = pd.DataFrame(all_candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df

# Obter dados de BTC/USDT em candles de 5 minutos nos últimos 3 meses
df = get_candles_last_3_months('BTC/USDT', '5m')

# Calcular RSI
def calculate_rsi(df, period=14):
    delta = df['close'].diff()
    gain = delta.clip(lower=0).rolling(window=period).mean()
    loss = -delta.clip(upper=0).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Calcular Bandas de Bollinger
def calculate_bollinger_bands(df, period=20, std_dev=3):
    """
    Calcula as Bandas de Bollinger com desvio padrão ajustável.
    """
    df['bollinger_middle'] = df['close'].rolling(window=period).mean()
    df['bollinger_upper'] = df['bollinger_middle'] + std_dev * df['close'].rolling(window=period).std()
    df['bollinger_lower'] = df['bollinger_middle'] - std_dev * df['close'].rolling(window=period).std()
    return df

# Adicionar RSI e Bandas de Bollinger ao DataFrame
df['rsi'] = calculate_rsi(df, period=14)
df = calculate_bollinger_bands(df, period=20, std_dev=3)

# Gerar sinais de compra e venda
def generate_signals(df, rsi_filter=50):
    """
    Gera sinais baseados nas Bandas de Bollinger, com filtro de RSI.
    """
    df['signal'] = 0
    # Sinal de compra: preço toca ou ultrapassa a banda inferior, RSI abaixo do filtro
    df.loc[
        (df['close'] <= df['bollinger_lower']) &
        (df['rsi'] < rsi_filter),
        'signal'
    ] = 1  # Compra

    # Sinal de venda: preço toca ou ultrapassa a banda superior, RSI acima do filtro
    df.loc[
        (df['close'] >= df['bollinger_upper']) &
        (df['rsi'] > rsi_filter),
        'signal'
    ] = -1  # Venda

    return df

# Gerar sinais
df = generate_signals(df, rsi_filter=50)

# Estratégia de backtest
def backtest_strategy(df, initial_balance=10000, stop_loss_pct=0.02):
    """
    Executa o backtest baseado em sinais gerados, aplicando stop-loss e take-profit.
    """
    balance = float(initial_balance)
    position = 0
    entry_price = 0
    df['portfolio_value'] = balance
    trades = 0
    correct_trades = 0

    for i in range(1, len(df)):
        # Comprar
        if df['signal'].iloc[i] == 1 and position == 0:
            position = 1
            entry_price = df['close'].iloc[i]
            trades += 1
            print(f"Compra em {df.index[i]} - Preço: {entry_price}")

        # Verificar posição ativa
        elif position == 1:
            # Verificar condições de stop-loss
            trade_return_low = (df['low'].iloc[i] - entry_price) / entry_price
            if trade_return_low <= -stop_loss_pct:
                balance *= 1 - stop_loss_pct
                position = 0
                print(f"Venda (Stop Loss) em {df.index[i]} - Preço: {entry_price * (1 - stop_loss_pct):.2f} - Saldo: {balance:.2f}")

            # Take profit até atingir a média das bandas
            elif df['close'].iloc[i] >= df['bollinger_middle'].iloc[i]:
                trade_return = (df['close'].iloc[i] - entry_price) / entry_price
                balance *= 1 + trade_return
                position = 0
                correct_trades += 1
                print(f"Venda (Take Profit) em {df.index[i]} - Preço: {df['close'].iloc[i]:.2f} - Retorno: {trade_return:.4f} - Saldo: {balance:.2f}")

        # Atualizar o valor do portfólio
        df.loc[df.index[i], 'portfolio_value'] = balance

    return df, trades, correct_trades

# Executar o backtest
df, total_trades, successful_trades = backtest_strategy(df, stop_loss_pct=0.02)

# Calcular lucro total
initial_balance = 10000
final_balance = df['portfolio_value'].iloc[-1]
total_profit = final_balance - initial_balance

# Mostrar os resultados
print(f"Número total de trades: {total_trades}")
print(f"Trades bem-sucedidos: {successful_trades}")
if total_trades > 0:
    print(f"Taxa de acerto: {successful_trades / total_trades * 100:.2f}%")
print(f"Lucro total: {total_profit:.2f}")

# Plotar os resultados
plt.figure(figsize=(14, 7))
plt.plot(df.index, df['close'], label='Preço de Fechamento')
plt.plot(df.index, df['bollinger_middle'], label='Banda Média', linestyle='--')
plt.plot(df.index, df['bollinger_upper'], label='Banda Superior', linestyle='--', color='red')
plt.plot(df.index, df['bollinger_lower'], label='Banda Inferior', linestyle='--', color='blue')
plt.plot(df.index, df['portfolio_value'], label='Valor do Portfólio')
plt.legend()
plt.show()
